In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, date_format, expr, floor, to_date,date_sub,trunc, month, rand
from pyspark.sql.types import IntegerType
from datetime import date
import pyspark.sql.functions as F
from dateutil.relativedelta import relativedelta
import datetime
import time
from pyspark.sql import Window

spark = SparkSession.builder \
    .appName("Ascend_DPD_Processing") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.9.1,org.apache.iceberg:iceberg-aws-bundle:1.9.1") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.autoBroadcastJoinThreshold","-1") \
    .config("spark.sql.join.preferSortMergeJoin", "true") \
    .config("spark.sql.adaptive.localShuffleReader.enabled", "true")\
    .config("spark.sql.optimizer.dynamicPartitionPruning.enabled", "true")\
    .config("spark.network.timeout", "800s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2") \
    .config("spark.sql.defaultCatalog", "ascend") \
    .config("spark.sql.catalog.ascend", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.ascend.warehouse", "s3://lf-test-1-bucket/ascend-dpd") \
    .config("spark.sql.catalog.ascend.type", "hadoop") \
    .config("spark.sql.catalog.ascend.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

def generate_iceberg_schema():
    spark.sql("""create table if not exists ascenddb.accounts_all (
                        CONS_ACCT_KEY BIGINT,
                        ACCT_BAL_AM BIGINT,
                        ACCT_CLOSED_DT DATE,
                        ACCT_CREDIT_EXT_AM BIGINT,
                        ACCT_DT DATE,
                        ACCT_OPEN_DT DATE,
                        ACCT_PYMT_STAT_CD VARCHAR(255),
                        ACCT_PYMT_STAT_DTL_CD VARCHAR(255),
                        ACCT_STAT_CD VARCHAR(255),
                        ACCT_TYPE_DTL_CD VARCHAR(255),
                        ACTUAL_PYMT_AM BIGINT,
                        ASSET_CLASS_CD VARCHAR(255),
                        BUREAU_MBR_ID VARCHAR(255),
                        CASH_LIMIT_AM BIGINT,
                        COLLATERAL_AM BIGINT,
                        COLLATERAL_CD  VARCHAR(255),
                        DAYS_PAST_DUE_CT INT,
                        HI_CREDIT_AM BIGINT,
                        INTEREST_RATE FLOAT,
                        INTEREST_RATE_4IN FLOAT,
                        LAST_PYMT_DT DATE,
                        NEXT_SCHD_PYMT_AM BIGINT,
                        ORIG_LOAN_AMT BIGINT,
                        ORIG_PYMT_DUE_DT DATE,
                        PAST_DUE_AM BIGINT,
                        PORT_TYPE_CD  VARCHAR(255),
                        PRINCIPAL_WRITE_OFF_AM BIGINT,
                        PYMT_TERMS_CD  VARCHAR(255),
                        PYMT_TERMS_DTL_CD  VARCHAR(255),
                        SCHD_PYMT_DT DATE,
                        SETTLED_AM BIGINT,
                        SUIT_FILED_WILFUL_DEF_STAT_CD  VARCHAR(255),
                        TOTAL_WRITE_OFF_AM BIGINT,
                        WO_SETTLED_STAT_CD  VARCHAR(255),
                        WRITE_OFF_AM BIGINT,
                        WRITE_OFF_DT DATE,

                        MSUBID VARCHAR(255),
                        CREDITLIMITAM BIGINT,
                        BALANCE_AM BIGINT,
                        BALANCE_DT DATE,
                        DFLTSTATUSDT DATE,
                        RESPONSIBILITY_CD VARCHAR(255),
                        CHARGEOFFAM BIGINT,
                        EMI_AMT BIGINT,
                        TENURE BIGINT,
                        PAYMENTRATINGCD VARCHAR(255),
                        PINCODE VARCHAR(255)

                        )
                        USING ICEBERG
                            PARTITIONED BY (months(ACCT_DT))""")

    print("Created schema for accounts_all")

    spark.sql("""
    CREATE TABLE if not exists ascenddb.summary(
                        CONS_ACCT_KEY BIGINT,
                        BUREAU_MBR_ID VARCHAR(255),
                        PORT_TYPE_CD VARCHAR(255),
                        ACCT_TYPE_DTL_CD VARCHAR(255),
                        ACCT_OPEN_DT DATE,
                        ORIG_LOAN_AMT BIGINT,
                        ACCT_CLOSED_DT DATE,
                        PYMT_TERMS_CD VARCHAR(255),
                        PYMT_TERMS_DTL_CD VARCHAR(255),
                        ACCT_DT DATE,
                        DPD_GRID STRING,
                        ACCT_STAT_CD VARCHAR(255),
                        ACCT_PYMT_STAT_CD VARCHAR(255),
                        ACCT_PYMT_STAT_DTL_CD VARCHAR(255),
                        ACCT_CREDIT_EXT_AM BIGINT,
                        ACCT_BAL_AM BIGINT,
                        PAST_DUE_AM BIGINT,
                        ACTUAL_PYMT_AM BIGINT,
                        LAST_PYMT_DT DATE,
                        SCHD_PYMT_DT DATE,
                        NEXT_SCHD_PYMT_AM BIGINT,
                        INTEREST_RATE FLOAT,
                        COLLATERAL_CD VARCHAR(255),
                        ORIG_PYMT_DUE_DT DATE,
                        WRITE_OFF_DT DATE,
                        WRITE_OFF_AM BIGINT,
                        ASSET_CLASS_CD VARCHAR(255),
                        DAYS_PAST_DUE INT,
                        DAYS_PAST_DUE_CT INT,
                        HI_CREDIT_AM BIGINT,
                        CASH_LIMIT_AM BIGINT,
                        COLLATERAL_AM BIGINT,
                        TOTAL_WRITE_OFF_AM BIGINT,
                        PRINCIPAL_WRITE_OFF_AM BIGINT,
                        SETTLED_AM BIGINT,
                        INTEREST_RATE_4IN FLOAT,
                        SUIT_FILED_WILFUL_DEF_STAT_CD VARCHAR(255),
                        WO_SETTLED_STAT_CD VARCHAR(255),
              
                        MSUBID VARCHAR(255),
                        CREDITLIMITAM BIGINT,
                        BALANCE_AM BIGINT,
                        BALANCE_DT DATE,
                        DFLTSTATUSDT DATE,
                        RESPONSIBILITY_CD VARCHAR(255),
                        CHARGEOFFAM BIGINT,
                        EMI_AMT BIGINT,
                        TENURE BIGINT,
                        PAYMENTRATINGCD VARCHAR(255),
                        PINCODE VARCHAR(255)          
        )
    USING iceberg
    PARTITIONED BY (months(ACCT_DT))""")

    print("Created schema for summary")

    spark.sql("""
    CREATE TABLE if not exists ascenddb.latest_dpd_summary(
        CONS_ACCT_KEY INT,
        ACCT_DT DATE, 
        DPD_GRID STRING
        )
    USING iceberg""")

    print("Created schema for latest_dpd_summary")

def dummy_data_creation():
    num_accounts = 720000000
    # months_needed = 335  # e.g., 10 years
    months_needed = 5  # e.g., 10 years
    total_records = num_accounts * months_needed

    # Step 1: Create DataFrame for all combinations
    df = spark.range(0, total_records)

    # Step 2: Generate account keys and month indices
    df = df.withColumn("CONS_ACCT_KEY", (col("id") / months_needed).cast("int") + 1) \
        .withColumn("month_index", (col("id") % months_needed).cast("int"))

    # Step 3: Generate date for each month (15th of month starting from Jan 2025)
    start_date = datetime.date(2018, 1, 15)
    dates = [start_date + relativedelta(months=i) for i in range(months_needed)]
    date_map = spark.createDataFrame([(i, d.strftime("%m/%d/%Y")) for i, d in enumerate(dates)],
                                    ["month_index", "ACCT_DT"])

    # Step 4: Join to get ACCT_DT
    df = df.join(date_map, on="month_index", how="left")

    # Step 5: Add DPD (random or deterministic)
    df = df.withColumn("DAYS_PAST_DUE_CT", ((col("CONS_ACCT_KEY") + col("month_index")) % 100 + 1))


    # Add dummy columns
    df = df.withColumn("BUREAU_MBR_ID", expr("concat('MBR', CONS_ACCT_KEY)")) \
           .withColumn("ACCT_BAL_AM", (rand() * 100000).cast("bigint")) \
           .withColumn("ACCT_CLOSED_DT", to_date(lit("2025-12-31"))) \
           .withColumn("ACCT_CREDIT_EXT_AM", (rand() * 50000).cast("bigint")) \
           .withColumn("ACCT_OPEN_DT", to_date(lit("2018-01-01"))) \
           .withColumn("ACCT_PYMT_STAT_CD", lit("A")) \
           .withColumn("ACCT_PYMT_STAT_DTL_CD", lit("A1")) \
           .withColumn("ACCT_STAT_CD", lit("O")) \
           .withColumn("ACCT_TYPE_DTL_CD", lit("T1")) \
           .withColumn("ACTUAL_PYMT_AM", (rand() * 10000).cast("bigint")) \
           .withColumn("ASSET_CLASS_CD", lit("STD")) \
           .withColumn("CASH_LIMIT_AM", (rand() * 20000).cast("bigint")) \
           .withColumn("COLLATERAL_AM", (rand() * 15000).cast("bigint")) \
           .withColumn("COLLATERAL_CD", lit("C1")) \
           .withColumn("HI_CREDIT_AM", (rand() * 120000).cast("bigint")) \
           .withColumn("INTEREST_RATE", (rand() * 15).cast("float")) \
           .withColumn("INTEREST_RATE_4IN", (rand() * 15).cast("float")) \
           .withColumn("LAST_PYMT_DT", to_date(lit("2025-06-30"))) \
           .withColumn("NEXT_SCHD_PYMT_AM", (rand() * 8000).cast("bigint")) \
           .withColumn("ORIG_LOAN_AMT", (rand() * 100000).cast("bigint")) \
           .withColumn("ORIG_PYMT_DUE_DT", to_date(lit("2025-01-15"))) \
           .withColumn("PAST_DUE_AM", (rand() * 5000).cast("bigint")) \
           .withColumn("PORT_TYPE_CD", lit("PT1")) \
           .withColumn("PRINCIPAL_WRITE_OFF_AM", (rand() * 3000).cast("bigint")) \
           .withColumn("PYMT_TERMS_CD", lit("M")) \
           .withColumn("PYMT_TERMS_DTL_CD", lit("M1")) \
           .withColumn("SCHD_PYMT_DT", to_date(lit("2025-07-15"))) \
           .withColumn("SETTLED_AM", (rand() * 2000).cast("bigint")) \
           .withColumn("SUIT_FILED_WILFUL_DEF_STAT_CD", lit("N")) \
           .withColumn("TOTAL_WRITE_OFF_AM", (rand() * 4000).cast("bigint")) \
           .withColumn("WO_SETTLED_STAT_CD", lit("Y")) \
           .withColumn("WRITE_OFF_AM", (rand() * 3500).cast("bigint")) \
           .withColumn("WRITE_OFF_DT", to_date(lit("2025-05-31"))) \
           .withColumn("CREDITLIMITAM", (rand() * 4000).cast("bigint")) \
           .withColumn("BALANCE_AM", (rand() * 4000).cast("bigint")) \
           .withColumn("CHARGEOFFAM", (rand() * 4000).cast("bigint")) \
           .withColumn("EMI_AMT", (rand() * 4000).cast("bigint")) \
           .withColumn("TENURE", (rand() * 4000).cast("bigint")) \
           .withColumn("MSUBID", lit("MSUB")) \
           .withColumn("RESPONSIBILITY_CD", lit("PT1")) \
           .withColumn("PAYMENTRATINGCD", lit("PT1")) \
           .withColumn("PINCODE", lit("687562")) \
           .withColumn("BALANCE_DT", to_date(lit("2025-05-31"))) \
           .withColumn("DFLTSTATUSDT", to_date(lit("2025-05-31")))

    # Final selection
    # df = df.select("CONS_ACCT_KEY", "ACCT_DT", "DAYSPASTDUE")


    df = df.select(
        "CONS_ACCT_KEY",
        "BUREAU_MBR_ID",
        "PORT_TYPE_CD",
        "ACCT_TYPE_DTL_CD",
        "ACCT_OPEN_DT",
        "ORIG_LOAN_AMT",
        "ACCT_CLOSED_DT",
        "PYMT_TERMS_CD",
        "PYMT_TERMS_DTL_CD",
        "ACCT_DT",
        "ACCT_STAT_CD",
        "ACCT_PYMT_STAT_CD",
        "ACCT_PYMT_STAT_DTL_CD",
        "ACCT_CREDIT_EXT_AM",
        "ACCT_BAL_AM",
        "PAST_DUE_AM",
        "ACTUAL_PYMT_AM",
        "LAST_PYMT_DT",
        "SCHD_PYMT_DT",
        "NEXT_SCHD_PYMT_AM",
        "INTEREST_RATE",
        "COLLATERAL_CD",
        "ORIG_PYMT_DUE_DT",
        "WRITE_OFF_DT",
        "WRITE_OFF_AM",
        "ASSET_CLASS_CD",
        "DAYS_PAST_DUE_CT",
        "HI_CREDIT_AM",
        "CASH_LIMIT_AM",
        "COLLATERAL_AM",
        "TOTAL_WRITE_OFF_AM",
        "PRINCIPAL_WRITE_OFF_AM",
        "SETTLED_AM",
        "INTEREST_RATE_4IN",
        "SUIT_FILED_WILFUL_DEF_STAT_CD",
        "WO_SETTLED_STAT_CD",
        "MSUBID",
        "CREDITLIMITAM",
        "BALANCE_AM",
        "BALANCE_DT",
        "DFLTSTATUSDT",
        "RESPONSIBILITY_CD",
        "CHARGEOFFAM",
        "EMI_AMT",
        "TENURE",
        "PAYMENTRATINGCD",
        "PINCODE"
    )


    # Verify uniqueness
    print("Expected Unique Keys:", df.count())
    # print("Distinct (CONS_acct_KEY, ACCT_DT):", df.select("CONS_acct_KEY", "ACCT_DT").distinct().count())

    print(f"Existing Partitions: {df.rdd.getNumPartitions()}")

    # df = df.repartition(6)
    df = df.withColumn("ACCT_DT",to_date(col("ACCT_DT"), "MM/dd/yyyy"))

    df.printSchema()

    # print(f"New Partitions: {stage_df.rdd.getNumPartitions()}")
    # Save to Parquet
    df.write\
    .format("iceberg")\
    .mode("overwrite")\
    .option("compression", "snappy")\
    .saveAsTable("ascenddb.accounts_all")

    df.explain()
    print("Data generated and saved to accounts_all")

def scenario_creation():
    queries = [
        "DELETE from ascenddb.dpd_data where CONS_ACCT_KEY = 336 AND ACCT_DT = '2018-04-15'",
        "DELETE FROM ascenddb.dpd_data WHERE CONS_ACCT_KEY = 337 AND ACCT_DT = '2018-01-15'",
        "DELETE FROM ascenddb.dpd_data WHERE CONS_ACCT_KEY = 337 AND ACCT_DT = '2018-04-15'",
        "DELETE FROM ascenddb.dpd_data WHERE CONS_ACCT_KEY = 337 AND ACCT_DT = '2018-05-15'",
    ]

    for query in queries:
        spark.sql(query)    
    
    print("Scenario Created")

def generate_dpd_grid():
    start_month = datetime.date(2018, 1, 1)
    end_month = datetime.date(2019, 10, 1)  # Adjust end date as needed

    month = start_month

    while month <= end_month:
        next_month = (month.replace(day=28) + datetime.timedelta(days=4)).replace(day=1)
        # print(f"Next Month: {next_month}")
        
        print(f"Processing month: {month} ...")

        summary_column_list = "CONS_ACCT_KEY, BUREAU_MBR_ID, PORT_TYPE_CD, ACCT_TYPE_DTL_CD, ACCT_OPEN_DT, ORIG_LOAN_AMT, ACCT_CLOSED_DT, PYMT_TERMS_CD, PYMT_TERMS_DTL_CD, ACCT_DT, ACCT_STAT_CD, ACCT_PYMT_STAT_CD, ACCT_PYMT_STAT_DTL_CD, ACCT_CREDIT_EXT_AM, ACCT_BAL_AM, PAST_DUE_AM, ACTUAL_PYMT_AM, LAST_PYMT_DT, SCHD_PYMT_DT, NEXT_SCHD_PYMT_AM, INTEREST_RATE, COLLATERAL_CD, ORIG_PYMT_DUE_DT, WRITE_OFF_DT, WRITE_OFF_AM, ASSET_CLASS_CD, DAYS_PAST_DUE_CT, HI_CREDIT_AM, CASH_LIMIT_AM, COLLATERAL_AM, TOTAL_WRITE_OFF_AM, PRINCIPAL_WRITE_OFF_AM, SETTLED_AM, INTEREST_RATE_4IN, SUIT_FILED_WILFUL_DEF_STAT_CD, WO_SETTLED_STAT_CD"

        # Load current month data
        current_df = spark.sql(f"""
            SELECT {summary_column_list}
            FROM ascenddb.accounts_all
            WHERE ACCT_DT >= DATE '{month}' AND ACCT_DT < DATE '{next_month}'
        """)
        
        # Load only latest record per account (tiny table)
        latest_prev = spark.read.table("ascenddb.latest_dpd_summary").withColumnRenamed("ACCT_DT", "ACCT_DT_prev")
        
        # Join (no longer need to scan historical months)
        merged_df = current_df.join(latest_prev, on="CONS_ACCT_KEY", how="left")
        
        # Continue with DPD_GRID generation as before
        merged_df = merged_df.withColumn(
            "MONTH_DIFF",
            (F.month("ACCT_DT") - F.month("ACCT_DT_prev")) +
            (F.year("ACCT_DT") - F.year("ACCT_DT_prev")) * 12
        ).withColumn(
            "FILLER_ARRAY",
            F.when(F.col("MONTH_DIFF") > 1, F.expr("transform(sequence(1, MONTH_DIFF - 1), x -> '?')"))
            .otherwise(F.array())
        ).withColumn(
            "Merged_DPD_Array",
            F.concat(
                F.array(F.col("DAYS_PAST_DUE_CT")),
                F.col("FILLER_ARRAY"),
                F.when(F.col("DPD_GRID").isNotNull(), F.split(F.col("DPD_GRID"), "~")).otherwise(F.array())
            )
        )
        
        # Pad & trim as usual
        merged_df = merged_df.withColumn(
            "DPD_Array_Trimmed",
            F.when(
                F.size("Merged_DPD_Array") >= 36,
                F.slice("Merged_DPD_Array", 1, 36)
            ).otherwise(
                F.concat(F.col("Merged_DPD_Array"),
                        F.array([F.lit("?") for _ in range(35)])
                )
            )
        ).withColumn(
            "DPD_GRID",
            F.concat_ws("~", "DPD_Array_Trimmed")
        )

        merged_df = merged_df.withColumn("DAYS_PAST_DUE",F.col("DAYS_PAST_DUE_CT"))
        
        # Save historical output (same as before)

        merged_df.select( "CONS_ACCT_KEY", "BUREAU_MBR_ID", "PORT_TYPE_CD", "ACCT_TYPE_DTL_CD", "ACCT_OPEN_DT", "ORIG_LOAN_AMT", "ACCT_CLOSED_DT", "PYMT_TERMS_CD", "PYMT_TERMS_DTL_CD", "ACCT_DT", "DPD_GRID", "ACCT_STAT_CD", "ACCT_PYMT_STAT_CD", "ACCT_PYMT_STAT_DTL_CD", "ACCT_CREDIT_EXT_AM", "ACCT_BAL_AM", "PAST_DUE_AM", "ACTUAL_PYMT_AM", "LAST_PYMT_DT", "SCHD_PYMT_DT", "NEXT_SCHD_PYMT_AM", "INTEREST_RATE", "COLLATERAL_CD", "ORIG_PYMT_DUE_DT", "WRITE_OFF_DT", "WRITE_OFF_AM", "ASSET_CLASS_CD", "DAYS_PAST_DUE", "DAYS_PAST_DUE_CT", "HI_CREDIT_AM", "CASH_LIMIT_AM", "COLLATERAL_AM", "TOTAL_WRITE_OFF_AM", "PRINCIPAL_WRITE_OFF_AM", "SETTLED_AM", "INTEREST_RATE_4IN", "SUIT_FILED_WILFUL_DEF_STAT_CD", "WO_SETTLED_STAT_CD" )\
        .write.format("iceberg").mode("append").saveAsTable("ascenddb.summary")


        # merged_df.select("CONS_ACCT_KEY", "ACCT_DT", "DAYS_PAST_DUE", "DPD_GRID") \
        #     .write.format("iceberg").mode("append").saveAsTable("ascenddb.summary")

        # Load current month processed data
        current_month_df = merged_df.select("CONS_ACCT_KEY", "ACCT_DT", "DPD_GRID")

        # Revert name of ACCT_DT_prev
        latest_prev = spark.read.table("ascenddb.latest_dpd_summary")
        
        # Merge: Take latest between old and new per account
        merged_latest = latest_prev.union(current_month_df) \
            .withColumn("rn", F.row_number().over(
                Window.partitionBy("CONS_ACCT_KEY").orderBy(F.col("ACCT_DT").desc())
            )) \
            .filter("rn = 1") \
            .drop("rn")
        
        # Save merged latest back
        merged_latest.write.mode("overwrite").saveAsTable("ascenddb.latest_dpd_summary")
        
        month = next_month    

generate_iceberg_schema()
dummy_data_creation()
scenario_creation()
generate_dpd_grid()



### Test Snapshot Based Single Execution

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, date_format, expr, floor, to_date,date_sub,trunc, month, rand
from pyspark.sql.types import IntegerType
from datetime import date
import pyspark.sql.functions as F
from dateutil.relativedelta import relativedelta
import datetime
import time
from pyspark.sql import Window

start_time = time.time()

spark = SparkSession.builder \
    .appName("Ascend_DPD_Processing") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.9.1,org.apache.iceberg:iceberg-aws-bundle:1.9.1") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.autoBroadcastJoinThreshold","-1") \
    .config("spark.sql.join.preferSortMergeJoin", "true") \
    .config("spark.sql.adaptive.localShuffleReader.enabled", "true")\
    .config("spark.sql.optimizer.dynamicPartitionPruning.enabled", "true")\
    .config("spark.network.timeout", "800s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2") \
    .config("spark.sql.defaultCatalog", "ascend") \
    .config("spark.sql.catalog.ascend", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.ascend.warehouse", "s3://lf-test-1-bucket/ascend-dpd") \
    .config("spark.sql.catalog.ascend.type", "hadoop") \
    .config("spark.sql.catalog.ascend.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

def generate_bureau_trade():

    snapshot_month = datetime.date(2020, 12, 1)
    snapshot_end_date = (snapshot_month.replace(day=28) + datetime.timedelta(days=4)).replace(day=1)
    snapshot_start_date = snapshot_month.replace(day=1)

    max_months=36

    end_date = (snapshot_month.replace(day=28) + datetime.timedelta(days=4)).replace(day=1)
    start_date = (snapshot_month - relativedelta(months=36)).replace(day=1)


    result_df = spark.sql(f"""
        WITH monthly_data AS (
            SELECT 
                CONS_ACCT_KEY AS ACCT_KEY,
                ACCT_DT AS SNAPSHOT_DT,
                DAYS_PAST_DUE_CT AS DAYSPASTDUE,
                DATE_TRUNC('month', ACCT_DT) as month_year
            FROM ascenddb.accounts_all
            WHERE 
            -- CONS_ACCT_KEY = 336 AND
            ACCT_DT >= DATE '{start_date}'
            AND ACCT_DT < DATE '{end_date}'
        ),
        
        -- Create a map of month -> DPD for easy lookup
        month_dpd_map AS (
            SELECT 
                ACCT_KEY,
                MAP_FROM_ARRAYS(
                    COLLECT_LIST(month_year),
                    COLLECT_LIST(DAYSPASTDUE)
                ) as month_to_dpd_map,
                MIN(month_year) as first_month,
                MAX(month_year) as last_month
            FROM monthly_data
            GROUP BY ACCT_KEY
        ),
        
        -- Join back to get arrays for each row
        array_builder AS (
            SELECT 
                md.ACCT_KEY,
                md.SNAPSHOT_DT,
                md.DAYSPASTDUE,
                md.month_year,
                mm.month_to_dpd_map,
                mm.first_month,
                -- Calculate months from first to current
                CAST(MONTHS_BETWEEN(md.month_year, mm.first_month) AS INT) as months_from_start,
                -- Build complete array with gaps filled
                TRANSFORM(
                    SEQUENCE(0, CAST(MONTHS_BETWEEN(md.month_year, mm.first_month) AS INT)),
                    i -> COALESCE(
                        mm.month_to_dpd_map[ADD_MONTHS(mm.first_month, i)],
                        '?'
                    )
                ) as complete_history
            FROM monthly_data md
            JOIN month_dpd_map mm ON md.ACCT_KEY = mm.ACCT_KEY
        ),
        
        final_arrays AS (
            SELECT 
                ACCT_KEY,
                SNAPSHOT_DT,
                DAYSPASTDUE,
                month_year,
                -- Reverse (most recent first) and limit/pad
                CONCAT_WS("~",
                            CASE 
                                WHEN SIZE(complete_history) > {max_months} THEN
                                    SLICE(REVERSE(complete_history), 1, {max_months})
                                ELSE
                                    CONCAT(
                                        REVERSE(complete_history),
                                        ARRAY_REPEAT('?', {max_months} - SIZE(complete_history))
                                    )
                            END) as DPD_GRID
            FROM array_builder
        )
        
        SELECT 
            ACCT_KEY,
            --SNAPSHOT_DT,
            --DAYSPASTDUE,
            DPD_GRID
        FROM final_arrays
        WHERE SNAPSHOT_DT >= DATE '{snapshot_start_date}' AND SNAPSHOT_DT < DATE '{snapshot_end_date}'
        ORDER BY ACCT_KEY, month_year
    """)

    assoc_df = spark.sql("""SELECT 
                                CONS_ACCT_KEY AS ACCT_KEY,
                                CAST(crc32(CAST(CAST(PIN AS STRING) AS BINARY)) AS STRING) AS CUSTOMER_ID 
                         FROM ascenddb.assoc_a""")
    
    result_df = result_df.join(assoc_df, on = 'ACCT_KEY', how="left")

    snapshot_df = spark.sql(f"""
                                SELECT 
                                    CONS_ACCT_KEY AS ACCT_KEY,
                                    ACCT_TYPE_DTL_CD AS ACCTTYPECD,
                                    ACCT_OPEN_DT AS OPEN_DT,
                                    ORIG_LOAN_AMT AS ORIGLOANAM,
                                    ACCT_DT AS SNAPSHOT_DT,
                                    DAYS_PAST_DUE_CT AS DAYSPASTDUE,
                                    ASSET_CLASS_CD AS ASSETCLASSCD,
                                    PAST_DUE_AM AS PASTDUEAM,
                                    ACCT_STAT_CD AS ACCOUNT_STATUS,
                                    ACCT_CLOSED_DT AS CLOSED_DT,
                                    ACTUAL_PYMT_AM AS ACTUALPAYMENTAM,
                                    LAST_PYMT_DT AS LASTPAYMENTDT,
                                    SUIT_FILED_WILFUL_DEF_STAT_CD AS SUITFILEDWILLFUL_DFLT,
                                    WO_SETTLED_STAT_CD AS WRITTENOFFANDSETTLEDSTATUS,
                                    WRITE_OFF_DT AS WRITEOFFSTATUS_DT,
                                    TOTAL_WRITE_OFF_AM AS TOTALWRITEOFF_AMT,
                                    PRINCIPAL_WRITE_OFF_AM AS PRINCIPLEWRITEOFF_AMT,
                                    SETTLED_AM AS SETTLED_AMT,
                                    INTEREST_RATE AS INTEREST_RATE,
                                    PORT_TYPE_CD AS PORTFOLIORATINGTYPE_CD,
                                    COLLATERAL_AM AS COLLATERAL_AMT,
                                    COLLATERAL_CD AS COLLATERAL_CD,
                                    MSUBID,
                                    CREDITLIMITAM,
                                    BALANCE_AM,
                                    BALANCE_DT,
                                    DFLTSTATUSDT,
                                    RESPONSIBILITY_CD,
                                    CHARGEOFFAM,
                                    EMI_AMT,
                                    TENURE,
                                    PAYMENTRATINGCD,
                                    PINCODE,
                                    DATE_TRUNC('month', ACCT_DT) as month_year
                                FROM ascenddb.accounts_all
                                WHERE 
                                -- CONS_ACCT_KEY = 336 AND
                                ACCT_DT >= DATE '{snapshot_start_date}'
                                AND ACCT_DT < DATE '{snapshot_end_date}'
                            """)

    result_df = snapshot_df.join(result_df, on = 'ACCT_KEY', how="left")


    result_df.select(
                        "CUSTOMER_ID",
                        "ACCT_KEY",
                        "ACCTTYPECD",
                        "OPEN_DT",
                        "ORIGLOANAM",
                        "SNAPSHOT_DT",
                        "DAYSPASTDUE",
                        "ASSETCLASSCD",
                        "PASTDUEAM",
                        "ACCOUNT_STATUS",
                        "CLOSED_DT",
                        "ACTUALPAYMENTAM",
                        "LASTPAYMENTDT",
                        "SUITFILEDWILLFUL_DFLT",
                        "WRITTENOFFANDSETTLEDSTATUS",
                        "WRITEOFFSTATUS_DT",
                        "TOTALWRITEOFF_AMT",
                        "PRINCIPLEWRITEOFF_AMT",
                        "SETTLED_AMT",
                        "INTEREST_RATE",
                        "PORTFOLIORATINGTYPE_CD",
                        "COLLATERAL_AMT",
                        "COLLATERAL_CD",
                        "DPD_GRID",
                        "MSUBID",
                        "CREDITLIMITAM",
                        "BALANCE_AM",
                        "BALANCE_DT",
                        "DFLTSTATUSDT",
                        "RESPONSIBILITY_CD",
                        "CHARGEOFFAM",
                        "EMI_AMT",
                        "TENURE",
                        "PAYMENTRATINGCD",
                        "PINCODE")\
    .write.mode("overwrite").parquet('s3://lf-test-1-bucket/ascend-dpd/bureau_trade.parquet', compression='snappy')

    print(f"Generated Bureau Trade Data for {snapshot_month}")

generate_bureau_trade()

end_time = time.time()
total_minutes = (end_time-start_time)/60
print(f"Execution Time :{total_minutes:.2f} minutes")


### Test Snapshot Incremental

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, date_format, expr, floor, to_date,date_sub,trunc, month, rand
from pyspark.sql.types import IntegerType
from datetime import date
import pyspark.sql.functions as F
from dateutil.relativedelta import relativedelta
import datetime
import time
from pyspark.sql import Window

spark = SparkSession.builder \
    .appName("Ascend_DPD_Processing") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.9.1,org.apache.iceberg:iceberg-aws-bundle:1.9.1") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.autoBroadcastJoinThreshold","-1") \
    .config("spark.sql.join.preferSortMergeJoin", "true") \
    .config("spark.sql.adaptive.localShuffleReader.enabled", "true")\
    .config("spark.sql.optimizer.dynamicPartitionPruning.enabled", "true")\
    .config("spark.network.timeout", "800s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2") \
    .config("spark.sql.defaultCatalog", "ascend") \
    .config("spark.sql.catalog.ascend", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.ascend.warehouse", "s3://lf-test-1-bucket/ascend-dpd") \
    .config("spark.sql.catalog.ascend.type", "hadoop") \
    .config("spark.sql.catalog.ascend.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.files.maxPartitionBytes", "268435456") \
    .config("spark.sql.parquet.block.size", "268435456") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")


def generate_dpd_grid():
    start_month = datetime.date(2018, 1, 1)
    end_month = datetime.date(2020, 12, 15)  # Adjust end date as needed

    month = start_month

    while month <= end_month:
        next_month = (month.replace(day=28) + datetime.timedelta(days=4)).replace(day=1)
        # print(f"Next Month: {next_month}")
        
        print(f"Processing month: {month} ...")

        summary_column_list = """CONS_ACCT_KEY, BUREAU_MBR_ID, PORT_TYPE_CD, ACCT_TYPE_DTL_CD, ACCT_OPEN_DT, ORIG_LOAN_AMT, ACCT_CLOSED_DT, PYMT_TERMS_CD, PYMT_TERMS_DTL_CD, ACCT_DT, ACCT_STAT_CD, ACCT_PYMT_STAT_CD, ACCT_PYMT_STAT_DTL_CD, ACCT_CREDIT_EXT_AM, ACCT_BAL_AM, PAST_DUE_AM, ACTUAL_PYMT_AM, LAST_PYMT_DT, SCHD_PYMT_DT, NEXT_SCHD_PYMT_AM, INTEREST_RATE, COLLATERAL_CD, ORIG_PYMT_DUE_DT, WRITE_OFF_DT, WRITE_OFF_AM, ASSET_CLASS_CD, DAYS_PAST_DUE_CT, HI_CREDIT_AM, CASH_LIMIT_AM, COLLATERAL_AM, TOTAL_WRITE_OFF_AM, PRINCIPAL_WRITE_OFF_AM, SETTLED_AM, INTEREST_RATE_4IN, SUIT_FILED_WILFUL_DEF_STAT_CD, WO_SETTLED_STAT_CD, MSUBID, CREDITLIMITAM, BALANCE_AM, BALANCE_DT, DFLTSTATUSDT, RESPONSIBILITY_CD, CHARGEOFFAM, EMI_AMT, TENURE, PAYMENTRATINGCD, PINCODE """
        
        # Load current month data
        current_df = spark.sql(f"""
            SELECT {summary_column_list}
            FROM ascenddb.accounts_all
            WHERE ACCT_DT >= DATE '{month}' AND ACCT_DT < DATE '{next_month}'
        """)
        
        # Load only latest record per account (tiny table)
        latest_prev = spark.read.table("ascenddb.latest_dpd_summary").withColumnRenamed("ACCT_DT", "ACCT_DT_prev")
        
        # Join (no longer need to scan historical months)
        merged_df = current_df.join(latest_prev, on="CONS_ACCT_KEY", how="left")
        
        # Continue with DPD_GRID generation as before
        merged_df = merged_df.withColumn(
            "MONTH_DIFF",
            (F.month("ACCT_DT") - F.month("ACCT_DT_prev")) +
            (F.year("ACCT_DT") - F.year("ACCT_DT_prev")) * 12
        ).withColumn(
            "FILLER_ARRAY",
            F.when(F.col("MONTH_DIFF") > 1, F.expr("transform(sequence(1, MONTH_DIFF - 1), x -> '?')"))
            .otherwise(F.array())
        ).withColumn(
            "Merged_DPD_Array",
            F.concat(
                F.array(F.col("DAYS_PAST_DUE_CT")),
                F.col("FILLER_ARRAY"),
                F.when(F.col("DPD_GRID").isNotNull(), F.split(F.col("DPD_GRID"), "~")).otherwise(F.array())
            )
        )
        
        # Pad & trim as usual
        merged_df = merged_df.withColumn(
            "DPD_Array_Trimmed",
            F.when(
                F.size("Merged_DPD_Array") >= 36,
                F.slice("Merged_DPD_Array", 1, 36)
            ).otherwise(
                F.concat(F.col("Merged_DPD_Array"),
                        F.array([F.lit("?") for _ in range(35)])
                )
            )
        ).withColumn(
            "DPD_GRID",
            F.concat_ws("~", "DPD_Array_Trimmed")
        )

        merged_df = merged_df.withColumn("DAYS_PAST_DUE",F.col("DAYS_PAST_DUE_CT"))
        
        # Save historical output (same as before)

        merged_df.select('CONS_ACCT_KEY', 'BUREAU_MBR_ID', 'PORT_TYPE_CD', 'ACCT_TYPE_DTL_CD', 'ACCT_OPEN_DT', 'ORIG_LOAN_AMT', 'ACCT_CLOSED_DT', 'PYMT_TERMS_CD', 'PYMT_TERMS_DTL_CD', 'ACCT_DT', 'ACCT_STAT_CD', 'ACCT_PYMT_STAT_CD', 'ACCT_PYMT_STAT_DTL_CD', 'ACCT_CREDIT_EXT_AM', 'ACCT_BAL_AM', 'PAST_DUE_AM', 'ACTUAL_PYMT_AM', 'LAST_PYMT_DT', 'SCHD_PYMT_DT', 'NEXT_SCHD_PYMT_AM', 'INTEREST_RATE', 'COLLATERAL_CD', 'ORIG_PYMT_DUE_DT', 'WRITE_OFF_DT', 'WRITE_OFF_AM', 'ASSET_CLASS_CD', 'DAYS_PAST_DUE_CT', 'HI_CREDIT_AM', 'CASH_LIMIT_AM', 'COLLATERAL_AM', 'TOTAL_WRITE_OFF_AM', 'PRINCIPAL_WRITE_OFF_AM', 'SETTLED_AM', 'INTEREST_RATE_4IN', 'SUIT_FILED_WILFUL_DEF_STAT_CD', 'WO_SETTLED_STAT_CD', 'MSUBID', 'CREDITLIMITAM', 'BALANCE_AM', 'BALANCE_DT', 'DFLTSTATUSDT', 'RESPONSIBILITY_CD', 'CHARGEOFFAM', 'EMI_AMT', 'TENURE', 'PAYMENTRATINGCD', 'PINCODE','DPD_GRID','DAYS_PAST_DUE' )\
        .write.format("iceberg").mode("append").saveAsTable("ascenddb.summary")


        # merged_df.select("CONS_ACCT_KEY", "ACCT_DT", "DAYS_PAST_DUE", "DPD_GRID") \
        #     .write.format("iceberg").mode("append").saveAsTable("ascenddb.summary")

        # Load current month processed data
        current_month_df = merged_df.select("CONS_ACCT_KEY", "ACCT_DT", "DPD_GRID")

        # Revert name of ACCT_DT_prev
        latest_prev = latest_prev.withColumnRenamed("ACCT_DT_prev", "ACCT_DT")
        
        # Merge: Take latest between old and new per account
        merged_latest = latest_prev.union(current_month_df) \
            .withColumn("rn", F.row_number().over(
                Window.partitionBy("CONS_ACCT_KEY").orderBy(F.col("ACCT_DT").desc())
            )) \
            .filter("rn = 1") \
            .drop("rn")
        
        # Save merged latest back
        merged_latest.write.mode("overwrite").saveAsTable("ascenddb.latest_dpd_summary")
        
        month = next_month    

# generate_iceberg_schema()
# dummy_data_creation()
# scenario_creation()
generate_dpd_grid()

